In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from ast_transformation.formula_generator import FormulaGenerator
from ast_transformation.formula_checker import FormulaChecker

from excel_utils import ExcelUtils

In [2]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_7'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [3]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [4]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [5]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [6]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [7]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [8]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [9]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [10]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [11]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [12]:
series_dict

{'Sheet1': [Series(series_id=SeriesId(sheet_name='Sheet1', series_header='App', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[None, None], values=['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
  Series(series_id=SeriesId(sheet_name='Sheet1', series_header='Category', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Category', formulas=[None, None], values=['ART_AND_DESIGN', 'ART_AND_DESIGN'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', value=None, value_type=None), series_length

In [13]:
series_mapping = SeriesMapper.map_series(series_dict)

In [14]:
series_mapping

{Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None): {Cell(column=1, row=2, coordinate=None, value=None, value_type=None): (0,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='App', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[None, None], values=['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>)),
  Cell(column=1, row=3, coordinate=None, value=None, value_type=None): (1,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='App', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[

In [15]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [16]:
series_list = [series for series in series_iterator if series.formulas != [None, None]]

In [17]:
import xlcalculator
import ast

from objects import Cell, HeaderLocation, CellRange

from excel_utils import ExcelUtils
from ast_transformation.formula_generator import SeriesIdLoader

class CellRangeImplementer:
    def __init__(self):
        pass

    @staticmethod
    def merge_cell_ranges(cell_ranges):
        min_row = min(cell_range.start_cell.row for cell_range in cell_ranges)
        min_column = min(cell_range.start_cell.column for cell_range in cell_ranges)
        max_row = max(cell_range.end_cell.row for cell_range in cell_ranges)
        max_column = max(cell_range.end_cell.column for cell_range in cell_ranges)

        start_cell = Cell(column=min_column, row=min_row, coordinate = ExcelUtils.get_coordinate_from_column_and_row(min_column, min_row))
        end_cell = Cell(column=max_column, row=max_row, coordinate = ExcelUtils.get_coordinate_from_column_and_row(max_column, max_row))

        return CellRange(start_cell=start_cell, end_cell=end_cell)

    @staticmethod
    def get_cell_range_from_series_tuple(series_tuple):
        series_ids_string, indexes = series_tuple
        series_start_index, series_end_index = indexes

        cell_ranges = []

        for series_id_string in series_ids_string:

            series_id = SeriesIdLoader.load_series_id_from_string(series_id_string)
            sheet_name = series_id.sheet_name

            series_list = series_dict.get(sheet_name)

            for series in series_list:
                if series.series_id == series_id:
                    cell_value = series.series_starting_cell
                    cell_row = cell_value.row
                    cell_column = cell_value.column

                    if series.header_location == HeaderLocation.TOP:
                        updated_start_cell_row = cell_row + series_start_index
                        updated_end_cell_row = cell_row + series_end_index
                        new_cell_range_value = CellRange(start_cell=Cell(row = updated_start_cell_row, column = cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row( cell_column, updated_start_cell_row)),
                                                        end_cell=Cell(row = updated_end_cell_row, column = cell_column + series_end_index, coordinate=ExcelUtils.get_coordinate_from_column_and_row( cell_column + series_end_index, updated_end_cell_row)))
                    elif series.header_location == HeaderLocation.LEFT:
                        updated_start_cell_column = cell_column + series_start_index
                        updated_end_cell_column = cell_column + series_end_index
                        new_cell_range_value = CellRange(start_cell=Cell(row = cell_row, column = updated_start_cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row( updated_start_cell_column, cell_row)),
                                                        end_cell=Cell(row = cell_row + series_end_index, column = updated_end_cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row( updated_end_cell_column, cell_row + series_end_index)))


                    else:
                        raise Exception("Header location is not valid")
                    cell_ranges.append(new_cell_range_value)
                    break

        
        cell_range =  CellRangeImplementer.merge_cell_ranges(cell_ranges)
        return cell_range

    def update_ast(self, ast):
        if isinstance(ast, xlcalculator.ast_nodes.RangeNode):
            return self.replace_range_node(ast)
        elif isinstance(ast, xlcalculator.ast_nodes.FunctionNode):
            return self.replace_function_node(ast)
        elif isinstance(ast, xlcalculator.ast_nodes.OperatorNode):
            return self.replace_operator_node(ast)
        return ast

    def replace_range_node(self, node):

        series_tuple = ast.literal_eval(node.tvalue)
        print(type(series_tuple))
        print(series_tuple)

        cell_range = CellRangeImplementer.get_cell_range_from_series_tuple(series_tuple)

        return xlcalculator.ast_nodes.RangeNode(
            xlcalculator.tokenizer.f_token(
                tvalue=cell_range, ttype="operand", tsubtype="range"
            ))
        

    def replace_function_node(self, node):
        modified_args = [self.update_ast(arg) for arg in node.args]
        modified_function_node = xlcalculator.ast_nodes.FunctionNode(node.token)
        modified_function_node.args = modified_args
        return modified_function_node

    def replace_operator_node(self, node):
        modified_left = self.update_ast(node.left) if node.left else None
        modified_right = self.update_ast(node.right) if node.right else None
        modified_operator_node = xlcalculator.ast_nodes.OperatorNode(node.token)
        modified_operator_node.left = modified_left
        modified_operator_node.right = modified_right
        return modified_operator_node

In [18]:
for series in series_list:
    formula_1 = series.formulas[0]
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_series = series_implementer.update_ast(formula_1_ast)

    formula_2 = series.formulas[1]
    formula_2_ast = FormulaParser.parse_formula(formula_2)
    formula_2_ast_series = series_implementer.update_ast(formula_2_ast)
    
    sheet_name = series.worksheet.sheet_name

    series_list = series_dict.get(sheet_name)

    ast_generator = FormulaGenerator.get_ast_generator(formula_1_ast_series, formula_2_ast_series, series_list)

    formulas_are_correct, formula_1_ast_new, formula_2_ast_new = FormulaChecker.check_formulas(ast_generator)

    cell_range_implementer = CellRangeImplementer()

    formula_1_ast_cell_ranges = cell_range_implementer.update_ast(formula_1_ast_new)
    print(formula_1_ast_cell_ranges)
    
    if not formulas_are_correct:
        print("formula_2_ast_series")
        print(formula_2_ast_series)
        print("formula_2_ast_new")
        print(formula_2_ast_new)
        raise Exception("Formulas are not correct")

<class 'tuple'>
(('Sheet1|App|1|1',), (0, 0))
<class 'tuple'>
(('Sheet1|Category|1|2',), (0, 0))
SUM(A2:A2, B2:B2)
<class 'tuple'>
(('Sheet1|Size|1|5',), (0, 0))
IF((E2:E2) = ("good"), "Yes", "No")
<class 'tuple'>
(('Sheet1|Category|1|2',), (None, None))


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'